In [1]:
#Carregando bibliotecas e lendo dataset
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
titanic = pd.read_csv("train.csv")
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#Dividindo os dados em categóricos e numéricos para processamento posterior
titanic_cat = titanic.select_dtypes(object)
titanic_num = titanic.select_dtypes(np.number)

In [3]:
titanic_cat.head()

,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,NaN,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,NaN,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,NaN,S


In [4]:
titanic_num.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
0,1,0,3,22.0,1,0,7.2500
1,2,1,1,38.0,1,0,71.2833
2,3,1,3,26.0,0,0,7.9250
3,4,1,1,35.0,1,0,53.1000
4,5,0,3,35.0,0,0,8.0500


In [5]:
#Removendo variáveis não informativas
titanic_cat.drop(['Name', 'Ticket'], axis=1, inplace=True)
#Avaliando valores nulos
titanic_cat.isnull().sum()

/home/audrey/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Sex           0
Cabin       687
Embarked      2
dtype: int64

In [6]:
#Substituindo a variável pela contagem máxima da categoria
titanic_cat.Cabin.fillna(titanic_cat.Cabin.value_counts().idxmax(), inplace=True)
titanic_cat.Embarked.fillna(titanic_cat.Embarked.value_counts().idxmax(), inplace=True)

/home/audrey/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [7]:
#Transformando categorias das variáveis nominais com labels numéricos
le = LabelEncoder()
titanic_cat = titanic_cat.apply(le.fit_transform)
titanic_cat.head()

,Sex,Cabin,Embarked
0,1,47,2
1,0,81,0
2,0,47,2
3,0,55,2
4,1,47,2


In [8]:
#Avaliando valores nulos variáveis numéricas
titanic_num.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Age            177
SibSp            0
Parch            0
Fare             0
dtype: int64

In [9]:
#Substituindo valores nulos pela média da variável idade
titanic_num.Age.fillna(titanic_num.Age.mean(), inplace=True)
titanic_num.isna().sum()

/home/audrey/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


PassengerId    0
Survived       0
Pclass         0
Age            0
SibSp          0
Parch          0
Fare           0
dtype: int64

In [10]:
#Removendo variáveis não informativas
titanic_num.drop(['PassengerId'], axis=1, inplace=True)
titanic_num.head()

/home/audrey/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Survived,Pclass,Age,SibSp,Parch,Fare
0,0,3,22.0,1,0,7.2500
1,1,1,38.0,1,0,71.2833
2,1,3,26.0,0,0,7.9250
3,1,1,35.0,1,0,53.1000
4,0,3,35.0,0,0,8.0500


In [11]:
#Recombinando dados categóricos com dados numéricos
titanic_final = pd.concat([titanic_cat, titanic_num], axis=1)
titanic_final.head()

,Sex,Cabin,Embarked,Survived,Pclass,Age,SibSp,Parch,Fare
0,1,47,2,0,3,22.0,1,0,7.2500
1,0,81,0,1,1,38.0,1,0,71.2833
2,0,47,2,1,3,26.0,0,0,7.9250
3,0,55,2,1,1,35.0,1,0,53.1000
4,1,47,2,0,3,35.0,0,0,8.0500


In [12]:
#Dividindo dados em treino e teste
X = titanic_final.drop(['Survived'], axis=1)
Y = titanic_final['Survived']

X_train = np.array(X[0:int(0.8*len(X))])
Y_train = np.array(Y[0:int(0.8*len(Y))])
X_test = np.array(X[int(0.8*len(X)):])
Y_test = np.array(Y[int(0.8*len(Y)):])
len(X_train), len(Y_train), len(X_test), len(Y_test)

(712, 712, 179, 179)

In [13]:
#Inicializando algoritmos
LR = LogisticRegression()
KNN = KNeighborsClassifier()
NB = GaussianNB()
LSVM = LinearSVC()
DT = DecisionTreeClassifier()

In [14]:
#Treinando algoritmos
LR_fit = LR.fit(X_train, Y_train)
KNN_fit = KNN.fit(X_train, Y_train)
NB_fit = NB.fit(X_train, Y_train)
LSVM_fit = LSVM.fit(X_train, Y_train)
DT_fit = DT.fit(X_train, Y_train)

/home/audrey/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/audrey/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [15]:
#Predizendo sobrevivência no conjunto de teste
LR_pred = LR_fit.predict(X_test)
KNN_pred = KNN_fit.predict(X_test)
NB_pred = NB_fit.predict(X_test)
LSVM_pred = LSVM_fit.predict(X_test)
DT_pred = DT_fit.predict(X_test)

In [16]:
#Avaliando acurácia dos modelos
print("Logistic Regression is %f percent accurate" %(accuracy_score(LR_pred, Y_test)*100))
print("KNN is %f percent accurate" %(accuracy_score(KNN_pred, Y_test)*100))
print("Naive Bayes is %f percent accurate" %(accuracy_score(NB_pred, Y_test)*100))
print("Linear SVMs is %f percent accurate" %(accuracy_score(LSVM_pred, Y_test)*100))
print("Decision Trees is %f percent accurate" %(accuracy_score(DT_pred, Y_test)*100))

Logistic Regression is 83.798883 percent accurate
KNN is 75.977654 percent accurate
Naive Bayes is 82.681564 percent accurate
Linear SVMs is 76.536313 percent accurate
Decision Trees is 81.005587 percent accurate
